Download Dataset

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d moeinkpr/snappfood-comments

!unzip snappfood-comments.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/moeinkpr/snappfood-comments
License(s): MIT
  0% 0.00/98.1M [00:00<?, ?B/s]
100% 98.1M/98.1M [00:00<00:00, 1.72GB/s]
Archive:  snappfood-comments.zip
  inflating: comments.csv            
  inflating: vendors.csv             


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 603, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 603 (delta 131), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (603/603), 199.38 KiB | 9.49 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Installing RAPIDS remaining 25.08 libraries
Using Python 3.12.11 environment at: /usr
Resolved 180 packages in 2.36s
Prepared 41 packages in 53.27s
Uninstalled 31 packages in 1.38s
Installed 41 packages in 826ms
 - bokeh==3.7.3
 + bokeh==3.6.3
 + cucim-cu12==25.8.0
 + cuda-bindings==12.9.3
 + cuda-pathfinder==1.3.0
 - cuda-python==12.6.2.post1
 + cuda-python==12.9.3
 - cudf-cu12==25.6.0 (from https://pypi.nvidia.com/cudf-cu12/cudf_cu12-25.6.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl)
 + cudf-cu12==25.8.0
 + cugraph-cu12==25.8.0
 - cuml-cu12==25.6.0
 + cuml-cu12==25.8.0
 - cuvs-cu12==25.6.1
 + cuvs-cu12==25.8.0
 + cu

Install hazm library

In [ ]:
#در این قسمت ابتدا سه لایبرری زیر را نصب می کنیم. بعد از اتمام نصب خط زیر را از کامنت در آورده و مجدد این سلول را ران کنید
#اگر چه بهتر است که کتابخانه hazm بر روی پایتون 3.10 نصب شود، اما می توان آن را بر روی 3.12 هم داشت.
# !pip uninstall -y hazm
!pip install numpy==1.26.4
!pip install hazm --no-deps
!pip install python-crfsuite fasttext-wheel gensim nltk flashtext

Found existing installation: hazm 0.10.0
Uninstalling hazm-0.10.0:
  Successfully uninstalled hazm-0.10.0
  Using cached hazm-0.10.0-py3-none-any.whl.metadata (11 kB)
Using cached hazm-0.10.0-py3-none-any.whl (892 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 18.2 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=8b4a10acc8250f074e137f80ea28ac244fdf3018a813b3568f2b977a2df4e866
  Stored in directory: /root/.cache/pip/wheels/8c/24/da/4d994d7a27cfc73a4e513a669fbeec4a71f871fe245a81977f
Suc

Import libraries

In [ ]:
import cudf
from cuml.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import re
import cudf
from hazm import Normalizer, Lemmatizer, word_tokenize
from cuml.cluster import KMeans
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

Pre processing


In [ ]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
comments_table = cudf.read_csv("comments.csv")
vendors_table = cudf.read_csv("vendors.csv")
# فیلد هایی که ارزش تحلیلی ندارند را حذف می کنیم تا جچم دیتا کاهش یابد و مدل سریع تر و بهینه تر آموزش یابد

comments_table = comments_table.drop(columns=["commentId","date","sender","rating","customerId","feeling","status","foods","replies"], errors="ignore")
vendors_table = vendors_table.drop(columns=["id","highlight","description","address","rating","title"], errors="ignore")
# دو جدول را بر اساس کد رستوران مرج می کنیم

merged_df = comments_table.merge(vendors_table, on="code", how="left")
# در دیتاست فقط رستوران ها رو انتخاب می کنیم و بعد از آن ستون vendorType را حذف می کنیم

merged_df = merged_df[merged_df["vendorType"] == "RESTAURANT"]
merged_df = merged_df.drop(columns=["vendorType"])

# دو ستون که مربط به کامنت ها بودند را مرج می کنیم تا همه کامنت ها را بررسی کنیم
# در ستون deliveryComment مقدار زیادی فیلد خالی وجود دارد
# برای این که در توسعه مدل اختلالی  وجود نیاید آن را با یک رشته خالی پر می کنیم

merged_df["deliveryComment"] = merged_df["deliveryComment"].fillna("")
merged_df["commentText"] = merged_df["commentText"].fillna("")

merged_df["text_raw"] = merged_df["commentText"] + " " + merged_df["deliveryComment"]
merged_df = merged_df.drop(columns=["commentText", "deliveryComment"])

# چون هدف ما این است که بصورت هفتگی روند رستوران ها را بررسی کنیم ستونی که مربوط به زمان ایجاد کامنت هست را به سال و هفته تقسیم می کنیم

merged_df["createdDate"] = cudf.to_datetime(merged_df["createdDate"])
merged_df["week"] = merged_df["createdDate"].dt.isocalendar().week.astype("int32")
merged_df["year"] = merged_df["createdDate"].dt.year.astype("int32")


Encoding

In [ ]:
# سفارش ها به چندین صورت به کاربر می رسد. به همین دلیل آن ها را انکود می کنیم تا حجم دیتا کاهش یابد

le = LabelEncoder()
merged_df["expeditionType_encoded"] = le.fit_transform(merged_df["expeditionType"].astype("str"))
merged_df = merged_df.drop(columns=["expeditionType"])

merged_df = merged_df.dropna(subset=["rate"])

merged_df["rate"] = merged_df["rate"].astype("uint8")


Data type casting

In [ ]:
#با استفاده از این تابع همه داده ها را بررسی کرده و تا جایی که امکان دارد آن ها را کاهش حجم می دهیم

def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


merged_df=auto_data_type(merged_df)

merged_df.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 1078372 entries, 16 to 1290548
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   createdDate             1078372 non-null  datetime64[ns]
 1   rate                    1078372 non-null  uint8
 2   code                    1078372 non-null  object
 3   commentCount            1078372 non-null  object
 4   text_raw                1078372 non-null  object
 5   week                    1078372 non-null  uint8
 6   year                    1078372 non-null  uint16
 7   expeditionType_encoded  1078084 non-null  uint8
dtypes: datetime64[ns](1), object(3), uint16(1), uint8(3)
memory usage: 151.6+ MB


Download comprehensive stop_words list

In [17]:
!kaggle datasets download -d alioraji/persian-stop-words
!unzip persian-stop-words.zip
with open("Persian_Stop_Words.txt", encoding='utf-8') as f:
    custom_stopwords = {line.strip() for line in f if line.strip()}

Dataset URL: https://www.kaggle.com/datasets/alioraji/persian-stop-words
License(s): other
persian-stop-words.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  persian-stop-words.zip
replace Persian_Stop_Words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Process on text and Categorize words

In [ ]:
#در این تابع استاندارد سازی و نرمال سازی متن را انجام می دهیم
def preprocess_text(text, stopwords):
    #متن نرمال سازی می شود
    text = normalizer.normalize(str(text))
    # کاراکتر های غیر فارسی حذف می شوند مانند علائم نگارشی یا اموجی ها
    text = re.sub(r"[^\u0600-\u06FF\s]", " ", text)
    # اعداد  از متن حذف می شود
    text = re.sub(r"\d+", "", text)
    # فاصله های اضافی حذف می شود
    text = re.sub(r"\s+", " ", text).strip()
    # متن را به یک سری توکن تبدیل می کنیم
    tokens = word_tokenize(text)
    # کلمات پر تکرار و بی معنی حذف می شود
    tokens = [t for t in tokens if t not in stopwords and len(t) > 2]
    # کلمات ریشه یابی می شود
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

# تابع را بر روی همه کامنت ها اعمال می شود. به دلیل آن که کتابخانه hazm بر روی cpu احرا می شود، این فرآیند یک مقدار زمان بر است.
text_series = merged_df["text_raw"].to_pandas()
text_clean = text_series.progress_apply(lambda x: preprocess_text(x, custom_stopwords))
merged_df["text_clean"] = cudf.Series(text_clean)

# هر جمله را به یک سری کلمات شکسته می شود تا آن ها را با استفاده از word2vec به بردار عددی تیدیل کنیم
sentences = [t.split() for t in text_clean if isinstance(t, str) and len(t) > 0]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, sg=1, workers=4, seed=42)

# پس از محاسبه بردار های عددی، 1500 کلمه اول را که پر تکرار بودند را انتخاب می کنیم
all_words = [w for s in sentences for w in s]
word_freq = (pd.Series(all_words).value_counts().reset_index().rename(columns={"index": "word", "word": "count"}))
common_words = [ w for w in word_freq["word"].head(1500) if w in w2v_model.wv]

#در این بخش، تعداد خوشه‌ها  را 20 تا در نظر گرفتیم و دسته ‌بندی‌های اصلی را 6 تا. دلیل این کار این است که الگوریتم KMeans به‌صورت خودکار ، فقط بر اساس شباهت عددی بین بردارهای کلمات، آن‌ها را در چند خوشه قرار می‌دهد تا واژه‌های مشابه از نظر معنا کنار هم بیایند. اما ۶ دسته‌ی اصلی مانند کیفیت، قیمت، ارسال و ...، دسته‌ های از پیش‌ تعریف ‌شده‌ای هستند که در نظر گرفته‌ شد (می توان دسته های بیشتری نیز اضافه کرد). بصورت کلی ، ابتدا مدل ۲۰ خوشه‌ می‌سازد تا کلمات نزدیک به هم را پیدا کند، سپس این خوشه‌ ها را در قالب ۶ دسته بندی طبقه‌ بندی می شود.

word_vectors = cudf.DataFrame([w2v_model.wv[w] for w in common_words])

kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
labels = kmeans.fit_predict(word_vectors)

clusters = {}
for w, lbl in zip(common_words, labels.to_pandas()):
    lbl = int(lbl)
    if lbl not in clusters:
        clusters[lbl] = []
    clusters[lbl].append(w)

categories = {
    "quality": ["کیفیت", "خوشمزه", "عالی", "بد", "راضی", "طعم"],
    "delivery": ["تحویل", "پیک", "ارسال", "دیر", "زود", "سریع"],
    "price": ["قیمت", "گرون", "ارزون", "هزینه", "پول"],
    "packaging": ["بسته", "بسته‌بندی", "جعبه", "ظرف", "کیسه"],
    "amount": ["حجم", "مقدار", "کم", "زیاد", "پرس"],
    "order_error": ["اشتباه", "جا", "فراموش", "غلط", "نداده"]
}


In [ ]:
for topic, words in categories.items():
    pattern = '|'.join(words)
    merged_df[f'flag_{topic}'] = merged_df['text_clean'].str.contains(pattern, regex=True).astype("uint8")

flags = list(categories.keys())

Train and test model

In [ ]:
# اگر امتیاز کامنت بیشتر از 4 باشد مثبت در نظر گرفته می شود و در غیر ای ن صورت منفی در نظر گرفته می شود. دلیل این کار این است که مدل بهتر بتواند متوجه شود چه کامنتی مثبت یا منفی بوده است و عوامل تاثیر گذار ذر آن کامنت  را درک کند.
merged_df["rating_class"] = (merged_df["rate"] >= 4).astype("int32")
flags = ["quality", "delivery", "price", "packaging", "amount", "order_error"]

results = []

# در این بخش صرفا رستوران هایی که تعداد کامنت های شان بیشتر از 30 عدد هست را انتخاب می کنیم
for code, group in merged_df.groupby("code"):
    if len(group) < 30:
        continue

# .اگر در رستورانی همه کامنت ها مثبت یا منفی بود نوع تقسیم بندی داده ها در ترین و تست مدل را عوض می کنیم.
# به عبارت دیگر چک می کنیم آیا کامنت مثبت و منفی هر دو برای آن رستوران وحود دارد یا خیر
    counts = group["rating_class"].value_counts()
    if len(counts) < 2 or counts.min() < 2:
        stratify_flag = False
    else:
        stratify_flag = True

    X = group[[f"flag_{f}" for f in flags]].astype("float32").to_pandas()
    y = group["rating_class"].to_pandas()

    if stratify_flag:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y )
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

    if len(np.unique(y_train)) < 2:
        continue

    model = XGBClassifier(
        n_estimators=1000,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        device="cuda",
        random_state=42,
        scale_pos_weight=len(y_train) / max(y_train.sum(), 1),
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

# سه عامل تاثیر گذار در امتیاز را انتخاب می کنیم
    importances = model.feature_importances_
    top3 = [f for f, v in sorted(zip(flags, importances), key=lambda x: -x[1])[:3]]

    results.append({ "code": code,"top_factors": ", ".join(top3),"accuracy": acc,"precision": prec,"recall": rec,"f1": f1 })


Evaluate metrics

In [21]:
mean_accuracy = np.mean([r["accuracy"] for r in results])
mean_precision = np.mean([r["precision"] for r in results])
mean_recall = np.mean([r["recall"] for r in results])
mean_f1 = np.mean([r["f1"] for r in results])

mean_table = [[
    round(mean_accuracy, 4),
    round(mean_precision, 4),
    round(mean_recall, 4),
    round(mean_f1, 4),
]]

print(tabulate(mean_table, headers=["Accuracy", "Precision", "Recall", "F1"], tablefmt="fancy_grid"))


╒════════════╤═════════════╤══════════╤════════╕
│   Accuracy │   Precision │   Recall │     F1 │
╞════════════╪═════════════╪══════════╪════════╡
│     0.8299 │      0.8321 │   0.9897 │ 0.9004 │
╘════════════╧═════════════╧══════════╧════════╛
